# Import


In [59]:
import os
import shutil
from pathlib import Path
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense, Input
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
import pandas as pd


In [60]:
# dotenv
try:
  from dotenv import load_dotenv
except ModuleNotFoundError:
  !pip install python-dotenv
  from dotenv import load_dotenv
load_dotenv()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


True

# Files


## Variables


In [61]:
data_folder = 'crops_diseases'
DATA_DIR = os.path.join('/',
                        'content',
                        data_folder,
                        )
LABELLED_DIR = os.path.join(
    DATA_DIR, 'BangladeshiCrops', 'BangladeshiCrops', 'Crop___Disease')
DRIVE_DIR = os.environ.get('DRIVE_DIR', '')


## Kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download nafishamoin/new-bangladeshi-crop-disease --unzip -p {data_folder}


100% 2.32G/2.33G [00:24<00:00, 154MB/s]
100% 2.33G/2.33G [00:24<00:00, 102MB/s]


## Google Colaboratory


In [ ]:
is_colab = False
try:
  from google.colab import drive
  drive.mount('/content/drive')
  is_colab = True
  Path(DRIVE_DIR).mkdir(parents=True, exist_ok=True)
except:
  pass


Mounted at /content/drive


## Clean Dirs


In [ ]:
for subdir, dirs, files in os.walk(DATA_DIR):
    for file in files:
        if file == '.DS_Store':
            os.remove(os.path.join(subdir, file))


# Preprocess


## Variables


In [ ]:
EPOCHS = 100
LEARNING_RATE = 1e-3
BS = 32
width = 200
height = 200
target_size = (width, height)
depth = 3
color_mode = 'rgb'
if depth == 1:
  color_mode = 'grayscale'
elif depth == 4:
  color_mode = 'rgba'


In [1]:
def get_slices(data_dir):
    data = []
    labels = []
    for subdir, dirs, files in os.walk(data_dir):
        for f in files:
            label = '__'.join(os.path.relpath(
                subdir, data_dir).split(os.path.sep))
            img_path = os.path.join(subdir, f)
            data.append(img_path)
            labels.append(label)
    return np.array(data), np.array(labels)


def preprocess_image(img_path, color_mode='rgb', target_size=(256, 256)):
    img = tf.keras.utils.load_img(img_path, color_mode=color_mode, target_size=target_size)
    img = tf.keras.utils.img_to_array(img)
    return img


## Augmentation


In [ ]:
fill_mode = 'nearest'
train_augmentation = Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip('horizontal_and_vertical'),
    layers.RandomTranslation(
        height_factor=0.15, width_factor=0.15, fill_mode=fill_mode),
    layers.RandomRotation(0.45, fill_mode=fill_mode),
    layers.RandomZoom(0.2, fill_mode=fill_mode),
    layers.RandomContrast(0.2),
])
val_augmentation = Sequential([
    layers.Rescaling(1./255),
])


### From Single image folder with shuffle


#### Slices


In [ ]:
images, labels = get_slices(LABELLED_DIR)


In [ ]:
columns = ['image', 'label']
column_image = columns[0]
column_label = columns[1]
df = pd.DataFrame(
    {column_image: images, column_label: labels}, columns=columns)
classes = sorted(df[column_label].unique())
n_classes = len(classes)


In [ ]:
# Preprocess labels
lb = LabelBinarizer()
lb.fit(classes)


LabelBinarizer()

In [ ]:
train_df, val_test_df = train_test_split(
    df, test_size=0.15, random_state=123, stratify=df[column_label])
val_df, test_df = train_test_split(
    val_test_df, test_size=0.4, random_state=123, stratify=val_test_df[column_label])


In [ ]:
print(f'Train: {len(train_df)}')
print(f'Validation: {len(val_df)}')
print(f'Test: {len(test_df)}')
print(f'Classes: {n_classes}')


Train: 11070
Validation: 1172
Test: 782
Classes: 14


#### ImageDataGenerator


In [ ]:
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=45,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.15,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
# )

# val_datagen = ImageDataGenerator(
#     rescale=1./255,
# )


##### flow


In [ ]:
# train_generator = train_datagen.flow(train_images, train_labels, batch_size=BS)
# val_generator = val_datagen.flow(val_images, val_labels, batch_size=BS)


##### flow_from_dataframe


In [ ]:
# train_generator = train_datagen.flow_from_dataframe(train_df,
#                                               x_col=columns[0],
#                                               y_col=columns[1],
#                                               target_size=target_size,
#                                               batch_size=BS,
# )

# val_generator = val_datagen.flow_from_dataframe(val_df,
#                                           x_col=columns[0],
#                                           y_col=columns[1],
#                                           target_size=target_size,
#                                           batch_size=BS,
# )

# test_generator = val_datagen.flow_from_dataframe(test_df,
#                                           x_col=columns[0],
#                                           y_col=columns[1],
#                                           target_size=target_size,
#                                           batch_size=BS,
# )


#### Tensorflow Data


##### from_tensor_slices


In [ ]:
# Dataset
cache_dir = 'ds_cache'
train_cache = os.path.join(cache_dir, 'train')
val_cache = os.path.join(cache_dir, 'val')
test_cache = os.path.join(cache_dir, 'test')
Path(cache_dir).mkdir(parents=True, exist_ok=True)

train_ds = tf.data.Dataset.from_tensor_slices((train_df[column_image], lb.transform(train_df[column_label]))).map(
    lambda x, y: (preprocess_image(x, color_mode, target_size), y), num_parallel_calls=tf.data.AUTOTUNE
).cache(
    train_cache
).map(
    lambda x, y: (train_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE
).shuffle(
    len(train_df[column_label])
).batch(
    BS, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_df[column_image], lb.transform(val_df[column_label]))).map(
    lambda x, y: (preprocess_image(x, color_mode, target_size), y), num_parallel_calls=tf.data.AUTOTUNE
).cache(
    val_cache
).map(
    lambda x, y: (val_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE
).shuffle(
    len(val_df[column_label])
).batch(
    BS, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_df[column_image], lb.transform(test_df[column_label]))).map(
    lambda x, y: (preprocess_image(x, color_mode, target_size), y), num_parallel_calls=tf.data.AUTOTUNE
).cache(
    test_cache
).map(
    lambda x, y: (val_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE
).batch(
    BS, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)


#### Samples


In [ ]:
# # Sample
# for sample_images, sample_labels in train_ds.take(1):
#   for i in range(10):
#     plt.figure()
#     plt.imshow(sample_images[i])
#     plt.title(classes[np.argmax(sample_labels[i])])
#   break


### From Single image folder without shuffle


#### Image Data Generator flow_from_directory


In [ ]:
# datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=45,
#     shear_range=0.15,
#     zoom_range=0.2,
#     vertical_flip=True,
#     horizontal_flip=True,
#     validation_split=0.2,
# )

# train_generator = datagen.flow_from_directory(
#     LABELLED_DIR,
#     target_size=target_size,
#     batch_size=BS,
#     subset='training',
# )

# val_generator = datagen.flow_from_directory(
#     LABELLED_DIR,
#     target_size=target_size,
#     batch_size=BS//2,
#     subset='validation',
# )


#### image_dataset_from_directory


In [ ]:
# # Split data into training and validation
# ds_kwargs = {'directory': LABELLED_DIR,
#            'batch_size': BS,
#            'image_size': target_size,
#            'validation_split': 0.15,
#            'seed': 123,
#            'label_mode': 'categorical',
#            }
# train_ds = tf.keras.preprocessing.image_dataset_from_directory(**ds_kwargs,
#                                                                subset='training',
#                                                                ).map(lambda x, y: (train_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
# val_ds = tf.keras.preprocessing.image_dataset_from_directory(**ds_kwargs,
#                                                                subset='validation',
#                                                                ).map(lambda x, y: (val_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)


### From Separated Training & Validation Folder


In [ ]:
# training_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=45,
#     shear_range=0.15,
#     zoom_range=0.2,
#     vertical_flip=True,
#     horizontal_flip=True,
# )

# val_datagen = ImageDataGenerator(
#     rescale=1./255,
# )

# train_generator = training_datagen.flow_from_directory(
#     TRAINING_DIR,
#     target_size=target_size,
#     batch_size=BS,
# )

# val_generator = val_datagen.flow_from_directory(
#     VALIDATION_DIR,
#     target_size=target_size,
#     batch_size=BS,
# )


## Assign same dataset variables


In [ ]:
try:
    train_ds = train_generator
    val_ds = val_generator
    test_ds = test_generator
except:
    pass


# Model


## Variables


In [ ]:
chan_dim = -1
input_shape = target_size + (depth,)
if K.image_data_format() == 'channels_first':
    input_shape = (depth,) + target_size
    chan_dim = 1


## Transfer Learning


In [ ]:
# Transfer Learning
base_model = keras.applications.EfficientNetB3(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
    pooling='max',
)


43950080/43941136 [==============================] - 0s 0us/step


In [ ]:
# base_model.trainable = False


In [ ]:
# base_model.summary()


### With transfer learning


In [ ]:
# # No additional Input layer
# inputs = base_model.input
# x = base_model(inputs, training=False)
# x = Dense(1024, activation=tf.nn.relu)(x)
# x = Dropout(0.2)(x)
# outputs = Dense(n_classes, activation=tf.nn.softmax)(x)
# model = Model(inputs=inputs, outputs=outputs)
# model.summary()


In [ ]:
# # Sequential - loss: 0.5479 - accuracy: 0.7955 - val_loss: 0.6703 - val_accuracy: 0.7899 - lr: 9.0000e-04
# model = Sequential([
#   base_model,
#   Conv2D(128, 3, padding='same'),
#   BatchNormalization(),
#   Activation(tf.nn.relu),
#   Dropout(0.2),
#   Conv2D(64, 3, padding='same', activation=tf.nn.relu),
#   GlobalAveragePooling2D(),
#   Dense(1024, activation=tf.nn.relu),
#   Dense(256, activation=tf.nn.relu),
#   Dense(n_classes, activation=tf.nn.softmax)
# ])
# model.summary()


In [ ]:
# # Sequential - loss: 0.5193 - accuracy: 0.8063 - val_loss: 0.6546 - val_accuracy: 0.7779 - lr: 9.0000e-05
# model = Sequential([
#   base_model,
#   Conv2D(128, 3, padding='same'),
#   BatchNormalization(),
#   Activation(tf.nn.relu),
#   Dropout(0.2),
#   Conv2D(64, 3, padding='same', activation=tf.nn.relu),
#   MaxPooling2D(2, 2),
#   Conv2D(64, 3, padding='same', activation=tf.nn.relu),
#   GlobalAveragePooling2D(),
#   # Dense(1024, activation=tf.nn.relu),
#   Dense(256, activation=tf.nn.relu),
#   Dense(n_classes, activation=tf.nn.softmax)
# ])
# model.summary()


In [ ]:
# # Sequential - loss: 0.5770 - accuracy: 0.7780 - val_loss: 0.6471 - val_accuracy: 0.7798 - lr: 3.0000e-04
# model = Sequential([
#   base_model,
#   Conv2D(128, 3, padding='same'),
#   BatchNormalization(axis=chan_dim),
#   Activation(tf.nn.relu),
#   Dropout(0.1),
#   Conv2D(64, 3, padding='same', activation=tf.nn.relu),
#   MaxPooling2D(2, padding='same'),
#   Conv2D(64, 3, padding='same'),
#   BatchNormalization(axis=chan_dim),
#   Activation(tf.nn.relu),
#   Dropout(0.1),
#   Conv2D(64, 3, padding='same', activation=tf.nn.relu),
#   GlobalAveragePooling2D(),
#   Dense(256, activation=tf.nn.relu),
#   Dense(n_classes, activation=tf.nn.softmax)
# ])
# model.summary()


In [ ]:
# # Sequential - loss: 0.5025 - accuracy: 0.8123 - val_loss: 0.6445 - val_accuracy: 0.7897 - lr: 9.0000e-05
# # Sequential - loss: 0.5629 - accuracy: 0.7832 - val_loss: 0.6472 - val_accuracy: 0.7738 - lr: 3.0000e-04
# # no decay - loss: 0.5694 - accuracy: 0.7794 - val_loss: 0.6564 - val_accuracy: 0.7718 - lr: 9.0000e-05
# model = Sequential([
#   base_model,
#   Conv2D(128, 3, padding='same'),
#   BatchNormalization(axis=chan_dim),
#   Activation(tf.nn.relu),
#   Dropout(0.1),
#   Conv2D(64, 3, padding='same', activation=tf.nn.relu),
#   MaxPooling2D(2, padding='same'),
#   # Conv2D(64, 3, padding='same'),
#   # BatchNormalization(axis=chan_dim),
#   # Activation(tf.nn.relu),
#   # Dropout(0.1),
#   Conv2D(64, 3, padding='same', activation=tf.nn.relu),
#   GlobalAveragePooling2D(),
#   Dense(256, activation=tf.nn.relu),
#   Dense(n_classes, activation=tf.nn.softmax)
# ])
# model.summary()


In [ ]:
model = Sequential([
    base_model,
    BatchNormalization(axis=chan_dim, momentum=0.99, epsilon=1e-3),
    Dense(256, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(l=16e-3),
          activity_regularizer=keras.regularizers.l1(6e-3), bias_regularizer=keras.regularizers.l1(6e-3)),
    Dropout(0.4, seed=123),
    Dense(n_classes, activation=tf.nn.softmax)
])
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 1536)             10783535  
                                                                 
 batch_normalization (BatchN  (None, 1536)             6144      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               393472    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 14)                3598      
                                                                 
Total params: 11,186,749
Trainable params: 11,096,374
Non-trainable params: 90,375
_____________________________________

## Optimizer & Compile


In [ ]:
# optimizer = Adam(learning_rate=LEARNING_RATE, decay=LEARNING_RATE / EPOCHS)
optimizer = optimizers.Adamax(learning_rate=LEARNING_RATE)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])


# Training


In [ ]:
BEST_MODEL = os.path.join('saved_model', 'best_model')
BEST_MODEL_H5 = BEST_MODEL + '.h5'
BEST_MODEL_SAVE_PATH = os.path.join(DRIVE_DIR, BEST_MODEL_H5)


def get_model_checkpoint(metrics_values={}):
    def evaluated_metrics(logs): return [
        logs[key] > val if 'accuracy' in key else logs[key] < val for key, val in metrics_values.items()]

    class MyModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):
        def on_epoch_end(self, epoch, logs):
            if False not in evaluated_metrics(logs):
                super().on_epoch_end(epoch, logs)
                global best_model
                best_model = self.model
                if is_colab:
                    shutil.copyfile(BEST_MODEL_H5, BEST_MODEL_SAVE_PATH)

    monitor = 'val_loss' if not metrics_values.keys(
    ) else next(iter(metrics_values.keys()))
    return MyModelCheckpoint(
        BEST_MODEL_H5,
        verbose=1,
        save_best_only=True,
        monitor=monitor)


In [ ]:
model_checkpoint = get_model_checkpoint({'val_loss': 0.7})


In [ ]:
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(factor=0.3,
                                                               patience=5,
                                                               min_lr=1e-5)


In [ ]:
early_stop_patience = max(((EPOCHS*0.2)//1, 10))
early_stopping = tf.keras.callbacks.EarlyStopping(patience=early_stop_patience)


In [ ]:
# # If we are looking for the best learning rate
# learning_rate_schedule = tf.keras.callbacks.LearningRateScheduler(
#     lambda epoch: LEARNING_RATE * 10**(epoch/2)
# )


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint, learning_rate_reduction, early_stopping],
    # # If we are looking for the best learning rate
    # callbacks=[learning_rate_schedule],
)


Epoch 1/100
346/346 [==============================] - 386s 555ms/step - loss: 5.7377 - accuracy: 0.8244 - val_loss: 6.4275 - val_accuracy: 0.1212 - lr: 0.0010
Epoch 2/100
346/346 [==============================] - 278s 529ms/step - loss: 2.4111 - accuracy: 0.9177 - val_loss: 3.3684 - val_accuracy: 0.3268 - lr: 0.0010
Epoch 3/100
346/346 [==============================] - 276s 529ms/step - loss: 1.1265 - accuracy: 0.9344 - val_loss: 3.5128 - val_accuracy: 0.1254 - lr: 0.0010
Epoch 4/100
346/346 [==============================] - 276s 529ms/step - loss: 0.6585 - accuracy: 0.9441 - val_loss: 1.7035 - val_accuracy: 0.5640 - lr: 0.0010
Epoch 5/100
346/346 [==============================] - 276s 529ms/step - loss: 0.5066 - accuracy: 0.9451 - val_loss: 5.3720 - val_accuracy: 0.0392 - lr: 0.0010
Epoch 6/100
346/346 [==============================] - 276s 528ms/step - loss: 0.4374 - accuracy: 0.9504 - val_loss: 21.1874 - val_accuracy: 0.0589 - lr: 0.0010
Epoch 7/100
346/346 [==================

# Result


Plot the train and val curve


In [ ]:
# # If we are looking for the best learning rate & to plot it
# plt.semilogx(history.history['lr'], history.history['loss'])
# plt.axis([1e-6, 1e-0, 0, 1.5])


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

# Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()

# Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()


# Test


In [ ]:
scores = model.evaluate(test_ds)
print(f"Test Accuracy: {scores[1]*100}")


In [ ]:
# from tensorflow.keras.models import load_model
# best_model = load_model(BEST_MODEL_H5)
scores = best_model.evaluate(test_ds)
print(f"Best Accuracy: {scores[1]*100}")


# Save and Zip


In [ ]:
MODEL_PATH = os.path.join('saved_model', 'model')
best_model.save(MODEL_PATH)
!zip -r model.zip {MODEL_PATH}
